# Map Chicago Traffic Accidents
- See Gallery for output html: https://github.com/zwrankin/folium_extensions/tree/master/gallery

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import folium

from folium_extensions.data.load_data import load_chicago_traffic_accident_brands_by_h3
from folium_extensions.h3 import h3df_to_geojson, add_h3_legend

In [3]:
df_h3 = load_chicago_traffic_accident_brands_by_h3()
df_h3.head()

/anaconda3/envs/folium_extensions/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3319: DtypeWarning: Columns (36,37,56,57,58,60,61,62,63,64,65,66,67,69) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,count,CHEVROLET,"TOYOTA MOTOR COMPANY, LTD.",FORD,NISSAN,HONDA,DODGE,HYUNDAI,JEEP,KIA MOTORS CORP,...,VOLKSWAGEN,BUICK,GENERAL MOTORS CORP.,LEXUS,MERCEDES-BENZ,BMW,MAZDA,PONTIAC,CADILLAC,INFINITI
h3,,,,,,,,,,,,,,,,,,,,,
0,3730,394.0,499.0,426.0,247.0,242.0,151.0,128.0,119.0,91.0,...,55.0,43.0,68.0,56.0,48.0,50.0,50.0,36.0,66.0,37.0
8826641903fffff,96,9.0,4.0,10.0,12.0,1.0,6.0,1.0,5.0,4.0,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,2.0,2.0
8826641905fffff,11,0.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
8826641907fffff,105,17.0,4.0,18.0,8.0,2.0,7.0,0.0,6.0,4.0,...,1.0,3.0,3.0,0.0,0.0,1.0,1.0,4.0,1.0,0.0
8826641909fffff,13,3.0,1.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
top_brands = df_h3.columns.tolist()
top_brands.remove('count')
top_brands

['CHEVROLET',
 'TOYOTA MOTOR COMPANY, LTD.',
 'FORD',
 'NISSAN',
 'HONDA',
 'DODGE',
 'HYUNDAI',
 'JEEP',
 'KIA MOTORS CORP',
 'CHRYSLER',
 'VOLKSWAGEN',
 'BUICK',
 'GENERAL MOTORS CORP.',
 'LEXUS',
 'MERCEDES-BENZ',
 'BMW',
 'MAZDA',
 'PONTIAC',
 'CADILLAC',
 'INFINITI']

## Choropleth

In [5]:
CENTER = [41.9,-87.7]
m = folium.Map(location=CENTER, zoom_start=11)

geojson = h3df_to_geojson(df_h3[['count']])

folium.Choropleth(
    geo_data=geojson,
    name='choropleth',
    data=df_h3.reset_index(),
    columns=['h3', 'count'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.5,
    line_opacity=0.1,
    legend_name='Accidents'
).add_to(m)

m.save('accident_choropleth.html')

## Categorical

In [6]:
import seaborn as sns
colors = sns.color_palette('deep', 10).as_hex() + sns.color_palette('bright', 10).as_hex()
COLOR_KEY = dict(zip(top_brands, colors))
COLOR_KEY

{'CHEVROLET': '#4c72b0',
 'TOYOTA MOTOR COMPANY, LTD.': '#dd8452',
 'FORD': '#55a868',
 'NISSAN': '#c44e52',
 'HONDA': '#8172b3',
 'DODGE': '#937860',
 'HYUNDAI': '#da8bc3',
 'JEEP': '#8c8c8c',
 'KIA MOTORS CORP': '#ccb974',
 'CHRYSLER': '#64b5cd',
 'VOLKSWAGEN': '#023eff',
 'BUICK': '#ff7c00',
 'GENERAL MOTORS CORP.': '#1ac938',
 'LEXUS': '#e8000b',
 'MERCEDES-BENZ': '#8b2be2',
 'BMW': '#9f4800',
 'MAZDA': '#f14cc1',
 'PONTIAC': '#a3a3a3',
 'CADILLAC': '#ffc400',
 'INFINITI': '#00d7ff'}

In [7]:
df_h3['top_brand_val'] = df_h3[top_brands].max(axis=1)
df_h3['top_brand_name'] = df_h3[top_brands].idxmax(axis=1)

In [8]:
m = folium.Map(location=CENTER, zoom_start=11)

geojson = h3df_to_geojson(df_h3)

def style_function(feature):
    # TODO - should I use the df_h3, or the geojson properties? The latter has some dtype issues (int32-64)
    row = df_h3.loc[feature['id']]
    color = COLOR_KEY[row['top_brand_name']]  # if row['top_brand_val'] > 0 else 'black'
    opacity = 0.7 if row['top_brand_val'] > 0 else 0
    return {
        'weight': 2,
        'opacity': opacity,
        'color': color,
        'fillColor': color,
        'fillOpacity': opacity, 
    }

folium.GeoJson(
    geojson,
    style_function=style_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['top_brand_name', 'top_brand_val'],
        aliases=['Vehicle Make', 'Value'],
    ),
).add_to(m)

m = add_h3_legend(m, COLOR_KEY, title='Accidents by Vehicle Make')

m.save('vehicle_brands_absolute.html')
m

In [9]:
zscore  = lambda x: (x - x.mean()) / (x.std())
zscores = df_h3[top_brands].copy()
for c in zscores.columns:
    zscores[c] = zscore(zscores[c])
    
df_h3['top_brand_val'] = zscores.max(axis=1)
df_h3['top_brand_name'] = zscores.idxmax(axis=1)

In [10]:
m = folium.Map(location=CENTER, zoom_start=11)

def style_function(feature):
    # TODO - should I use the df_h3, or the geojson properties? The latter has some dtype issues (int32-64)
    row = df_h3.loc[feature['id']]
    color = COLOR_KEY[row['top_brand_name']]  # if row['top_brand_val'] > 0 else 'black'
    opacity = 0.7 if row['top_brand_val'] > 0 else 0  # don't plot negative z-scores
    return {
        'weight': 2,
        'opacity': opacity,
        'color': color,
        'fillColor': color,
        'fillOpacity': opacity, 
    }

folium.GeoJson(
    geojson,
    style_function=style_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['top_brand_name', 'top_brand_val'],
        aliases=['Vehicle Make', 'Value'],
    ),
).add_to(m)

m = add_h3_legend(m, COLOR_KEY, title='Accidents by Vehicle Make')

m.save('vehicle_brands_relative.html')
m

# Use as layers

In [11]:
m = folium.Map(location=CENTER, zoom_start=11)

df_h3['top_brand_val'] = df_h3[top_brands].max(axis=1)
df_h3['top_brand_name'] = df_h3[top_brands].idxmax(axis=1)

def style_function(feature):
    row = df_h3.loc[feature['id']]
    color = COLOR_KEY[row['top_brand_name']]  # if row['top_brand_val'] > 0 else 'black'
    opacity = 0.7 if row['top_brand_val'] > 0 else 0
    return {
        'weight': 2,
        'opacity': opacity,
        'color': color,
        'fillColor': color,
        'fillOpacity': opacity, 
    }

fg = folium.FeatureGroup(name='absolute', show=True)
folium.GeoJson(
    geojson,
    style_function=style_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['top_brand_name', 'top_brand_val'],
        aliases=['Vehicle Make', 'Value'],
    ),
).add_to(fg)
m.add_child(fg)


df_h3['top_brand_z_val'] = zscores.max(axis=1)
df_h3['top_brand_z_name'] = zscores.idxmax(axis=1)

geojson = h3df_to_geojson(df_h3)
def style_function(feature):
    row = df_h3.loc[feature['id']]
    color = COLOR_KEY[row['top_brand_z_name']]  # if row['top_brand_val'] > 0 else 'black'
    opacity = 0.7 if row['top_brand_z_val'] > 0 else 0
    return {
        'weight': 2,
        'opacity': opacity,
        'color': color,
        'fillColor': color,
        'fillOpacity': opacity, 
    }
fg = folium.FeatureGroup(name='relative (z-score)', show=False)
folium.GeoJson(
    geojson,
    style_function=style_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['top_brand_z_name', 'top_brand_z_val'],
        aliases=['Vehicle Make', 'Value'],
    ),
).add_to(fg)
m.add_child(fg)

m.add_child(folium.LayerControl(collapsed=False))

m = add_h3_legend(m, COLOR_KEY, title='Accidents by Vehicle Make')

m.save('vehicle_brands.html')
m.save('../gallery/accidents_by_brand.html')